In [259]:
import psycopg2
from connection_data import *

### СОЗДАНИЕ ТАБЛИЦ В БАЗЕ ДАННЫХ

In [260]:
try:
    conn = psycopg2.connect(host=my_host, port=my_port, database=main_database, user=username, password=user_password)
    cur = conn.cursor()

    cur.execute("DROP TABLE entities;")
    cur.execute("DROP TABLE cases CASCADE;")
    cur.execute("DROP TABLE documents CASCADE;")
    print("Таблицы 'entities', 'cases' и 'documents' были успешно удалены")
    
    # создание таблицы, хранящей инф-цию о судебных делах
    cur.execute("CREATE TABLE IF NOT EXISTS cases \
                (id SERIAL PRIMARY KEY, \
                number VARCHAR(50) UNIQUE, \
                court VARCHAR(255), \
                participants TEXT);")
    print("Таблица 'cases' была успешно создана")

    # создание таблицы, хранящей инф-цию о юридических документах
    cur.execute("CREATE TABLE IF NOT EXISTS documents \
                (id SERIAL PRIMARY KEY, \
                title VARCHAR(255), \
                date DATE, \
                type VARCHAR(50), \
                case_id INTEGER REFERENCES cases(id));")
    print("Таблица 'documents' была успешно создана")
    
    # создание таблицы, хранящей именованные сущности из документов
    cur.execute("CREATE TABLE IF NOT EXISTS entities \
                (id SERIAL PRIMARY KEY, \
                name VARCHAR(255), \
                type VARCHAR(50), \
                document_id INTEGER REFERENCES documents(id));")
    print("Таблица 'entities' была успешно создана")

    conn.commit()
    cur.close
    conn.close()

except psycopg2.Error as e:
    print("[error] Ошибка подключения к базе данных Postges")
    print(e)
    

Таблицы 'entities', 'cases' и 'documents' были успешно удалены
Таблица 'cases' была успешно создана
Таблица 'documents' была успешно создана
Таблица 'entities' была успешно создана


### НАПОЛНЕНИЕ ТАБЛИЦ ДАННЫМИ

In [261]:
try:
    conn = psycopg2.connect(host=my_host, port=my_port, database=main_database, user=username, password=user_password)
    cur = conn.cursor()

    # добавление данных в таблицу cases
    cur.execute("INSERT INTO cases (number, court, participants) VALUES \
                ('СИП-162', 'Федеральная служба по интеллектуальной собственности', 'ООО \"Четыре лапы\", ООО\"ГРУМ\"'), \
                ('21АП-6475', 'Арбитражный суд города Севастополя', 'Геворгян Геворг Хачикович'), \
                ('А73-2806', 'Арбитражный суд Хабаровского края', 'ПАО \"ДЭК\", ФГАУ \"Росжилкомплекс\"'), \
                ('А56-14358', 'Арбитражный суд города Санкт-Петербурга и Ленинградской области', 'ГУ Управление Пенсионного Фонда РФ, ООО \"ПРОДТОРГ\"'), \
                ('СИП-184', 'Суд по интеллектуальным правам', 'Ленциг Акциенгезельшафт, ВИОРИКА-КОСМЕТИК Эс.А'), \
                ('А71-2265', 'Арбитражный суд Удмуртской Республики', 'ООО \"Спецавтохозяйство\", ООО \"Альянс\"'), \
                ('А82-3222', 'Арбитражный суд Ярославской области', 'Коржов Олег Игоревич'), \
                ('А84-767', 'Арбитражный суд города Севастополя', 'Роговская Кристина Евгеньевна'), \
                ('А81-651', 'Арбитражный суд Ямало-Ненецкого АО', 'ОАО \"Российские железные дороги\", ПАО \"Государственная транспортная лизинговая компания\"'), \
                ('А81-640', 'Арбитражный суд Ямало-Ненецкого АО', 'ПАО \"Ростелеком\", ООО \"Виват\"'), \
                ('08АП-5518', '8 арбитражный апелляционный суд', 'ПАО \"Ростелеком\", ООО \"Виват\"'), \
                ('Ф04-5118', 'Арбитражный суд Западно-Сибирского округа', 'ПАО \"Ростелеком\", ООО \"Виват\"');")
    print("Новые записи успешно добавлены в таблицу 'cases'")

    # добавление данных в таблицу documents
    cur.execute("INSERT INTO documents (title, date, type, case_id) VALUES \
                ('Решение суда 1-ой инстанции о частичном удовлетворении иска', '2023-10-04', 'Решение', 1), \
                ('Назначение дела к судебному разбирательству', '2023-06-27', 'Назначение', 1), \
                ('Решение суда 1-ой инстанции о признании гражданина банкротом', '2023-10-25', 'Решение', 2), \
                ('Определение суда 1-ой инстанции о прекращении производства по заявлению/жалобе', '2023-08-21', 'Определение', 2), \
                ('Постановление суда апелляционной инстанции оставить определение суда без изменения, жалобу без удовлетворения', '2024-10-18', 'Постановление', 2), \
                ('Решение суда 1-ой инстанции о взыскании денежных средств и возврате госпошлины', '2025-02-18', 'Решение', 3), \
                ('Назначение дела к судебному разбирательству', '2025-01-13', 'Назначение', 3), \
                ('Выдача судебного приказа (ст.229.6 АПК)', '2021-03-04', 'Судебный приказ', 4), \
                ('Определение суда 1-ой инстанции о прекращении производства по делу в связи с принятием отказа от заявленных требований', '2021-07-20', 'Определение', 5), \
                ('Определение суда 1-ой инстанции об оставлении искового заявления без рассмотрения', '2021-09-29', 'Определение', 6), \
                ('Определение суда 1-ой инстанции о принятии к производству заявления о признании должника банкротом', '2025-02-07', 'Определение', 8), \
                ('Решение суда 1-ой инстанции об отказе в иске', '2020-06-21', 'Решение', 9), \
                ('Решение суда 1-ой инстанции об удовлетворении ходатайства', '2020-05-20', 'Решение', 10), \
                ('Постановление суда апелляционной инстанции оставить решение суда без изменения', '2020-08-27', 'Постановление', 11), \
                ('Постановление суда кассационной инстанции оставить постановление апелляционной инстанции без изменения', '2020-12-25', 'Постановление', 12);")
    print("Новые записи успешно добавлены в таблицу 'documents'")
    
    # добавления данных в таблицу entities
    cur.execute("INSERT INTO entities (name, type, document_id) VALUES \
                ('Интеллектуальные права', 'Термин', 1), \
                ('Банкротство', 'Термин', 2), \
                ('Госпошлина', 'Термин', 3), \
                ('Судебный приказ', 'Документ', 4), \
                ('Интеллектуальные права', 'Термин', 5), \
                ('Банкротство', 'Термин', 7), \
                ('Банкротство', 'Термин', 8), \
                ('Ходатайство', 'Документ', 10);")
    print("Новые записи успешно добавлены в таблицу 'entities'")

    conn.commit()
    cur.close
    conn.close()

except psycopg2.Error as e:
    print("[error] Ошибка подключения к базе данных Postges")
    print(e)

Новые записи успешно добавлены в таблицу 'cases'
Новые записи успешно добавлены в таблицу 'documents'
Новые записи успешно добавлены в таблицу 'entities'


### ВЫПОЛНЕНИЕ SQL-ЗАПРОСОВ 

In [262]:
try:
    conn = psycopg2.connect(host=my_host, port=my_port, database=main_database, user=username, password=user_password)
    cur = conn.cursor()

    # получение всех документов типа "Постановление"
    cur.execute("SELECT * FROM documents WHERE type = 'Постановление';")

    print("Все документы типа 'Постановление':")
    for document in cur.fetchall():
        d_id, d_title, d_date, d_type, d_case_id = document
        print(f" id_{d_id} -- {d_title} -- {d_date} -- {d_type} -- {d_case_id}")

    # получение дела с номером "СИП-184"
    cur.execute("SELECT * FROM cases WHERE number = 'СИП-184';")
    c_id, c_number, c_court, c_participants = cur.fetchone()
    print("\nДело с номером 'СИП-184':")
    print(f" id_{c_id} -- {c_number} -- {c_court} -- {c_participants} ")

    # получение номеров всех дел, в которых участвует 'ПАО "Ростелеком"'
    cur.execute("SELECT number FROM cases WHERE participants LIKE '%ПАО \"Ростелеком\"%';")

    print('\nНомера всех дел, в которых участвует \'ПАО "Ростелеком"\':')
    for case in cur.fetchall():
        c_number = case[0]
        print(c_number, end="    ")

    # получение пар (документ, сущность), для которых индексы документов указаны в таблице с сущностями
    cur.execute("SELECT d.title, e.name \
                FROM documents d \
                JOIN entities e ON d.id = e.document_id;")
    
    print("\n\nПары (название документа, имя сущности), для которых индексы документов указаны в таблице с сущностями:")
    for document, entity in cur.fetchall():
        print(f" {document} -- {entity}")


    # получение пар (сущность, документ), связанных с делами Арбитражного суда города Севастополя
    cur.execute("SELECT e.name AS entity_name, d.title AS document_title \
                FROM entities e \
                JOIN documents d ON e.document_id = d.id \
                WHERE d.id IN (SELECT id FROM cases WHERE court = 'Арбитражный суд города Севастополя');")

    print("\nПары (имя сущности, название документа), связанных с делами Арбитражного суда города Севастополя:")
    for entity, document in cur.fetchall():
        print(f" {entity} -- {document}")

    # получение кол-ва документов по каждому делу
    cur.execute("SELECT c.number, COUNT(d.id) AS document_count \
                FROM cases c \
                LEFT JOIN documents d ON c.id = d.case_id \
                GROUP BY c.number;")

    print("\nКол-во документов каждому делу:")
    for c_number, count in cur.fetchall():
        print(f" {c_number} --> {count}")

    # получение всех дел с их документами, отсортированными по дате документа (от старых к свежим)
    # [ номер_дела -- название_суда ] --> [ название_документа -- дата_документа ]
    cur.execute("SELECT c.number, c.court, d.title, d.date \
                FROM cases c \
                LEFT JOIN documents d ON c.id = d.case_id \
                ORDER BY d.date ASC;")

    print("\nВсе дела с их документами, отсортированными по дате документа (от старых к свежим):")
    for case in cur.fetchall():
        c_number, c_court, d_title, d_date = case
        print(f" [ {c_number} -- {c_court} ] --> [ {d_title} -- {d_date} ] ")

    # получение всех дел с их документами, отсортированными по дате документа (от свежих к старым и не ранее 1 октября 2023)
    # [ номер_дела -- название_суда ] --> [ название_документа -- дата_документа ]
    cur.execute("SELECT c.number, c.court, d.title, d.date \
                FROM cases c \
                LEFT JOIN documents d ON c.id = d.case_id \
                WHERE d.date >= '2023-10-01' \
                ORDER BY d.date DESC;")

    print("\nВсе дела с их документами, отсортированными по дате документа (от свежих к старым и не ранее 1 октября 2023):")
    for case in cur.fetchall():
        c_number, c_court, d_title, d_date = case
        print(f" [ {c_number} -- {c_court} ] --> [ {d_title} -- {d_date} ] ")
    
    cur.close
    conn.close()

except psycopg2.Error as e:
    print("[error] Ошибка подключения к базе данных Postges")
    print(e)

Все документы типа 'Постановление':
 id_5 -- Постановление суда апелляционной инстанции оставить определение суда без изменения, жалобу без удовлетворения -- 2024-10-18 -- Постановление -- 2
 id_14 -- Постановление суда апелляционной инстанции оставить решение суда без изменения -- 2020-08-27 -- Постановление -- 11
 id_15 -- Постановление суда кассационной инстанции оставить постановление апелляционной инстанции без изменения -- 2020-12-25 -- Постановление -- 12

Дело с номером 'СИП-184':
 id_5 -- СИП-184 -- Суд по интеллектуальным правам -- Ленциг Акциенгезельшафт, ВИОРИКА-КОСМЕТИК Эс.А 

Номера всех дел, в которых участвует 'ПАО "Ростелеком"':
А81-640    08АП-5518    Ф04-5118    

Пары (название документа, имя сущности), для которых индексы документов указаны в таблице с сущностями:
 Решение суда 1-ой инстанции о частичном удовлетворении иска -- Интеллектуальные права
 Назначение дела к судебному разбирательству -- Банкротство
 Решение суда 1-ой инстанции о признании гражданина банкр

### ПОЛНОТЕКСТОВЫЕ ИНДЕКСЫ ДЛЯ ПОИСКА

In [263]:
try:
    conn = psycopg2.connect(host=my_host, port=my_port, database=main_database, user=username, password=user_password)
    cur = conn.cursor()

    # создание полнотекстового индекса для заголовков документов
    cur.execute("CREATE INDEX IF NOT EXISTS idx_documents_title \
                ON documents USING gin(to_tsvector('russian', title));")
    # создание полнотекстового индекса для участников дел
    cur.execute("CREATE INDEX IF NOT EXISTS idx_cases_participants \
                ON cases USING gin(to_tsvector('russian', participants));")
    
    # поиск всех документов, в названии которых есть слово 'банкрот'
    cur.execute("SELECT * FROM documents WHERE \
                to_tsvector('russian', title) @@ to_tsquery('банкрот');")

    print("Все документы, в названии которых есть слово 'банкрот':")
    for document in cur.fetchall():
        d_id, d_title, d_date, d_type, d_case_id = document
        print(f" id_{d_id} -- {d_title} -- {d_date} -- {d_type} -- {d_case_id}")
    
    # поиск всех документов, в названии которых есть слово 'производство'
    cur.execute("SELECT * FROM documents \
                WHERE to_tsvector('russian', title) @@ to_tsquery('производство');")

    print("\nВсе документы, в названии которых есть слово 'производство':")
    for document in cur.fetchall():
        d_id, d_title, d_date, d_type, d_case_id = document
        print(f" id_{d_id} -- {d_title} -- {d_date} -- {d_type} -- {d_case_id}")

    # поиск дел, в которых участвуют 'ООО'
    cur.execute("SELECT * FROM cases \
                WHERE to_tsvector('russian', participants) @@ to_tsquery('ООО');")

    print("\nВсе дела, участниками которых являются 'ООО':")
    for case in cur.fetchall():
        c_id, c_number, c_court, c_participants = case
        print(f" id_{c_id} -- {c_number} -- {c_court} -- {c_participants} ")

    # поиск всех дел, у которых в названии документов упоминается "решение"
    cur.execute("SELECT c.* FROM cases c \
                JOIN documents d ON d.case_id = c.id \
                WHERE to_tsvector('russian', d.title) @@ to_tsquery('решение');")

    print("\nВсе дела, в названии документов которых есть слово 'решение':")
    for case in cur.fetchall():
        c_id, c_number, c_court, c_participants = case
        print(f" id_{c_id} -- {c_number} -- {c_court} -- {c_participants} ")

    # поиск всех сущностей, соответствующих документам, в названии которых есть слово "банкрот"
    cur.execute("SELECT e.* FROM entities e \
                JOIN documents d ON e.document_id = d.id \
                WHERE to_tsvector('russian', d.title) @@ to_tsquery('банкрот');")

    print("\nВсе сущности, соответствующие документам, в названии которых есть слово 'банкрот':")
    for entity in cur.fetchall():
        e_id, e_name, e_type, e_document_id = entity
        print(f" id_{e_id} -- {e_name} -- {e_type} -- {e_document_id} ")

    # поиск всех дел, соответствующих документам, в названии которых есть слово "судебный"
    cur.execute("SELECT c.* FROM cases c \
                JOIN documents d ON c.id = d.case_id \
                WHERE to_tsvector('russian', d.title) @@ to_tsquery('судебный');")

    print("\nВсе дела, соответствующие документам, в названии которых есть слово 'судебный':")
    for case in cur.fetchall():
        c_id, c_number, c_court, c_participants = case
        print(f" id_{c_id} -- {c_number} -- {c_court} -- {c_participants} ")

    # поиск документов по нескольким ключевым словам
    cur.execute("SELECT * FROM documents \
                WHERE to_tsvector('russian', title) @@ to_tsquery('постановление | определение');")

    print("\nВсе документы, в названии которых есть слово 'постановление' ИЛИ 'определение':")
    for document in cur.fetchall():
        d_id, d_title, d_date, d_type, d_case_id = document
        print(f" id_{d_id} -- {d_title} -- {d_date} -- {d_type} -- {d_case_id}")
    
    cur.close
    conn.close()

except psycopg2.Error as e:
    print("[error] Ошибка подключения к базе данных Postges")
    print(e)

Все документы, в названии которых есть слово 'банкрот':
 id_3 -- Решение суда 1-ой инстанции о признании гражданина банкротом -- 2023-10-25 -- Решение -- 2
 id_11 -- Определение суда 1-ой инстанции о принятии к производству заявления о признании должника банкротом -- 2025-02-07 -- Определение -- 8

Все документы, в названии которых есть слово 'производство':
 id_4 -- Определение суда 1-ой инстанции о прекращении производства по заявлению/жалобе -- 2023-08-21 -- Определение -- 2
 id_9 -- Определение суда 1-ой инстанции о прекращении производства по делу в связи с принятием отказа от заявленных требований -- 2021-07-20 -- Определение -- 5
 id_11 -- Определение суда 1-ой инстанции о принятии к производству заявления о признании должника банкротом -- 2025-02-07 -- Определение -- 8

Все дела, участниками которых являются 'ООО':
 id_1 -- СИП-162 -- Федеральная служба по интеллектуальной собственности -- ООО "Четыре лапы", ООО"ГРУМ" 
 id_4 -- А56-14358 -- Арбитражный суд города Санкт-Петербур

### КОНТРОЛЬНЫЕ ВОПРОСЫ
#### 1. Что такое первичный ключ и зачем он используется?

*Первичный ключ* -- это уникальный идентификатор для каждой записи в таблице базы данных. Он обеспечивает уникальность каждой строки, что позволяет легко находить, обновлять и удалять записи.

**Характеристики:**
- для каждой записи в таблице он уникален,
- не может содержать пустые значения (NULL),
- обычно индексируется, что обеспечивает более быстрое выполнение операций поиска.

**Использование:**
- позволяет поддерживать целостность данных, предотвращая дублирование записей,
- упрощает связь между таблицами в реляционных базах данных (особенно в случае использования внешних ключей).

#### 2. Как создание индексов может ускорить процесс поиска?

Индексы работают как указатели на строки в таблице, позволяя базе данных быстро находить нужные записи без необходимости просматривать каждую строку.

**Преимущества:**
- быстрое выполнение запросов, даже на больших объёмах данных,
- оптимизация сортировки (ORDER BY) и фильтрации (WHERE),
- улучшение производительности операции соединения таблиц (JOIN).

#### 3. Какие преимущества даёт использование внешних ключей?
*Внешний ключ* -- поле (или набор полей) в одной таблице, которое ссылается на первичный ключ другой таблицы. Он используется для создания связей между таблицами и обеспечения целостности данных.

**Преимущества:**
- обеспечивает целотности данных (значения в одном столбце одной таблицы соответствуют значениям в другом столбце другой таблицы, т.е нельзя создать запись с несуществующим идентификатором),
- позволяет создать логические связи между таблицами,
- поддерживает каскадные операции (можно настроить автоматическое изменение связанных записей при изменении основной),
- позволяет лучше организовать структуру базы данных и упростить её проектирование в целом.